In [1]:
%matplotlib widget
import matplotlib.pyplot as plt


In [ ]:
plt.ioff()

In [ ]:
lines = plt.plot([1, 2, 3, 4])
plt.show()

In [ ]:
plt.ion()


In [7]:
lines[0].set_ydata([1, 2, 3,5])
# plt.draw_all()
# print(plt.gcf().stale, plt.gca().stale)
# plt.gcf().stale = False
# plt.gca().stale = False
# print(plt.gcf().stale, plt.gca().stale)

In [ ]:
plt.gcf().stale, plt.gca().stale 

In [ ]:
plt.gca().stale = True
plt.gcf().stale = True
plt.gcf().stale, plt.gca().stale 

In [ ]:
plt.gcf().stale, plt.gca().stale 

In [ ]:
# ipympl docs example showing how to get error messages
import ipywidgets as widgets
import numpy as np
plt.ion()
x = np.linspace(0,np.pi,200)
y = np.linspace(0,10,200)
X,Y = np.meshgrid(x,y)
parameter = np.linspace(-5,5)
example_image_stack = np.sin(X)[None,:,:]+np.exp(np.cos(Y[None,:,:]*parameter[:,None,None]))
with plt.ioff():
    fig = plt.figure()
im = plt.imshow(example_image_stack[0])

out = widgets.Output()
@out.capture()
def update(change):
    with out:
        if change['name'] == 'value':
            im.set_data(example_image_stack[change['new']])
            fig.canvas.draw_idle
    
    
slider = widgets.IntSlider(value=0, min=0, max=len(parameter)+10)
slider.observe(update)
display(widgets.VBox([slider, fig.canvas]))
display(out)

In [ ]:
# When using the `widget` backend from ipympl,
# fig.canvas is a proper Jupyter interactive widget, which can be embedded in
# an ipywidgets layout. See https://ipywidgets.readthedocs.io/en/stable/examples/Layout%20Templates.html

# One can bound figure attributes to other widget values.
from ipywidgets import AppLayout, FloatSlider

plt.ioff()

slider = FloatSlider(
    orientation='horizontal',
    description='Factor:',
    value=1.0,
    min=0.02,
    max=2.0
)

slider.layout.margin = '0px 30% 0px 30%'
slider.layout.width = '40%'

fig = plt.figure()
fig.canvas.header_visible = False
fig.canvas.layout.min_height = '400px'
plt.title('Plotting: y=sin({} * x)'.format(slider.value))

x = np.linspace(0, 20, 500)

lines = plt.plot(x, np.sin(slider.value * x))

def update_lines(change):
    plt.title('Plotting: y=sin({} * x)'.format(change.new))
    lines[0].set_data(x, np.sin(change.new * x))
    fig.canvas.draw()
    fig.canvas.flush_events()

slider.observe(update_lines, names='value')

AppLayout(
    center=fig.canvas,
    footer=slider,
    pane_heights=[0, 6, 1]
)

In [7]:
from numba import jit

@jit
def mandel(x, y, max_iters):
    """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
    """
    i = 0
    c = complex(x,y)
    z = 0.0j
    for i in range(max_iters):
        z = z*z + c
        if (z.real*z.real + z.imag*z.imag) >= 4:
            return i

    return 255

@jit
def create_fractal(min_x, max_x, min_y, max_y, iters):
    image = np.zeros((600, 600), dtype=np.uint8)
    height = image.shape[0]
    width = image.shape[1]

    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height
    for x in range(width):
        real = min_x + x * pixel_size_x
        for y in range(height):
            imag = min_y + y * pixel_size_y
            color = mandel(real, imag, iters)
            image[y, x] = color

    return image

In [260]:
import ipywidgets as widgets
range_slider = widgets.FloatRangeSlider(
    value=[5, 7.5],
    min=0,
    max=10.0,
    step=0.1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
widgets.link((range_slider, 'value'), (ax_traits, 'xlim'))
# range_slider.link(ax_traits, xlim='value')
range_slider

0 axis xlim event called (np.float64(5.0), np.float64(7.5))


FloatRangeSlider(value=(5.0, 7.5), continuous_update=False, description='Test:', max=10.0, readout_format='.1f…

In [261]:
plt.close('all')

In [219]:
ax_traits.xlim = (0, 2)
ax.get_xlim(),ax_traits.xlim

((np.float64(0.0), np.float64(2.0)), (np.float64(0.0), np.float64(2.0)))

In [216]:
ax_traits.ylim = (0, 2)
ax.get_ylim(),ax_traits.ylim

((np.float64(0.0), np.float64(2.0)), (np.float64(0.0), np.float64(2.0)))

In [215]:
plt.gca().set_xlim(0, 3)
ax.get_xlim(),ax_traits.xlim


((np.float64(0.0), np.float64(3.0)), (np.float64(0.0), np.float64(3.0)))

In [218]:
plt.gca().set_ylim(0, 3)
ax.get_ylim(),ax_traits.ylim


((np.float64(0.0), np.float64(3.0)), (np.float64(0.0), np.float64(3.0)))

In [129]:
ax.xaxis._set_lim

<bound method Axis._set_lim of <matplotlib.axis.XAxis object at 0x71f140f53190>>

In [99]:
(AxisTraitlet(ax)).trait_values()

{'xlim': (), 'ylim': (), 'zlim': ()}

In [4]:
import asyncio
from time import time

class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()
def throttle(wait):
    """ Decorator that prevents a function from being called
        more than once every wait period. """
    def decorator(fn):
        time_of_last_call = 0
        scheduled, timer = False, None
        new_args, new_kwargs = None, None
        def throttled(*args, **kwargs):
            nonlocal new_args, new_kwargs, time_of_last_call, scheduled, timer
            def call_it():
                nonlocal new_args, new_kwargs, time_of_last_call, scheduled, timer
                time_of_last_call = time()
                fn(*new_args, **new_kwargs)
                scheduled = False
            time_since_last_call = time() - time_of_last_call
            new_args, new_kwargs = args, kwargs
            if not scheduled:
                scheduled = True
                new_wait = max(0, wait - time_since_last_call)
                timer = Timer(new_wait, call_it)
                timer.start()
        return throttled
    return decorator

In [12]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.backend_bases
import matplotlib.pyplot as plt
%matplotlib widget
from collections.abc import Iterable
# import time
import math
from dynamic_figure import AxisTraitlet, pan_factory, zoom_factory, __clear
import ipywidgets as widgets

# maybe also accept a list of callables? and then update each one?
# limits can accepts widgets and update the limits of the axes
out = widgets.Output()
@out.capture()
def dynamic_figureF(callable, limits=None, ipywidgets_controls={}, **subplots_kwargs):
    with plt.ioff():
        figure, ax = plt.subplots(**subplots_kwargs)
        _ax = ax
        if isinstance(_ax, Iterable):
            from itertools import chain
            _ax = list(chain.from_iterable(_ax))
        else:
            # not iterable
            _ax = [ax]
        if limits is not None: # should check if iterable? should None where values don't exist?
            for a,lims in zip(_ax,limits):
                # TODO
                # lims should be iterable lims[0], lims[1]
                # lims[0 or 1] can be a widget or a 2-tuple(iterable)
                # lims [0 or 1][0 or 1] can be None, widget or float like value
                _a_traitlet = AxisTraitlet(a)
                if isinstance(lims[0][0], widgets.Widget):
                    widgets.link((lims[0][0], 'value'), (_a_traitlet, 'xlim'))
                else:
                    a.set_xlim(lims[0])
                if isinstance(lims[1][0], widgets.Widget):
                    widgets.link((lims[1][0], 'value'), (_a_traitlet, 'ylim'))
                else:
                    a.set_ylim(lims[1])

    # for a in _ax:
        # AxisTraitlet(a)
    @out.capture()
    def draw(callable):
        # try:
            callable(figure,ax)
        # except Exception as e:
            # raise e
    @throttle(0.05)
    def redraw(event: matplotlib.backend_bases.MouseEvent):
        update(callable)
    pan_factory(figure,listener=redraw)
    zoom_factory(figure, listener=redraw)
    for a in _ax:
        a.callbacks.connect('xlim_changed', redraw)
        a.callbacks.connect('ylim_changed', redraw)
        a.callbacks.connect('zlim_changed', redraw)
        
    last_time = float("-inf")
    @out.capture()
    def update(callable):

        # nonlocal last_time
        # find a better debouncer that works without threads?
        # this_time = time.time()
        # if this_time - last_time < 0.05:
            # return
        # last_time = this_time
        
        for a in _ax:
            # left,right = a.get_xlim()
            # bottom, top = a.get_ylim()
            # a.cla() # maybe overwrite this function? it emits an event from axis_set_lim https://github.com/matplotlib/matplotlib/blob/a254b687df97cda8c6affa37a1dfcf213f8e6c3a/lib/matplotlib/axes/_base.py#L1379
            __clear(a)
            # auto = False #??
            # a.yaxis._set_lim(bottom, top, emit=False, auto=auto)
            # a.xaxis._set_lim(left, right, emit=False, auto=auto)
        
        old_fig = plt.gcf()
        plt.figure(figure)
        draw(callable)
        plt.figure(old_fig)
        
    draw(callable)
    if plt.isinteractive():
        plt.show()
    return figure, ax

def callable(fig,ax, ipywidgets_controls={},limits=None, mouse_position=None):
    k = np.random.randn()*10
    plt.plot([1, 2, 3, k])
    ax[1,0].plot([1, 2, k, 3])
    ax[1,0].set_title("1,0")
    
    x0,x1,y0,y1 = ax[0,0].axis()
    xy = create_fractal(x0, x1, -y1, -y0, 15)
    ax[0,0].imshow(xy, extent= (x0, x1,y0 , y1))

    x0,x1,y0,y1 = ax[0,1].axis()
    xy = create_fractal(x0, x1, -y1, -y0, 15)
    ax[0,1].imshow(xy, extent= (x0, x1,y0 , y1))
   


import ipywidgets as widgets
range_slider = widgets.FloatRangeSlider(
    value=[-1, 1],
    min=-1,
    max=1.0,
    step=0.1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
# range_slider.link(ax_traits, xlim='value')

with plt.ioff():
    figure, ax = dynamic_figureF(callable,ncols=2,nrows=2,ipywidgets_controls={},limits = [[[range_slider],[None,None]],[[-2.5,1],[-1,1]],[[-2,15],[1,5]]])#,[[-2,2],[-2,2]]])
# ax_traits = AxisTraitlet(ax[0,0])
# widgets.link((range_slider, 'value'), (ax_traits, 'xlim'))
# range_slider
display(widgets.VBox([range_slider, figure.canvas]))
display(out)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Output()

In [283]:
def linked(axx):
    print(axx)
    ax[0,1].set_xlim(ax.get_xlim())
ax[0,0].callbacks.connect('xlim_changed', linked)

3

In [288]:
plt.close('all')

In [37]:
[item for item in sublist for sublist in ax]

NameError: name 'sublist' is not defined

In [ ]:
plt.show()